# Darcy equation

In this tutorial we present how to solve an evolutionary Darcy equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unkwons are the velocity $q$ and the pressure $p$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Let $(0,T)$ with $10=T>0$ be the overall simulation period. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1} {q} + \nabla p = {- \rho g \nabla y}\\
p_t + \nabla \cdot {q} = f
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$
with boundary conditions:
$$ p = 0 \text{ on } \partial_{top} \Omega \times (0,T] \qquad p = \rho g \text{ on } \partial_{bottom} \Omega \times (0,T] \qquad \nu \cdot q = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega \times (0,T] $$
and initial conditions:
$$ p|_{t=0} = (1-y) \rho g \text{ in } \Omega \qquad q|_{t=0} = 0 \text{ in } \Omega $$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
%load_ext Cython

In [2]:
import shutil
import os

import numpy as np
import scipy.sparse as sps
import scipy.integrate as integrate

import porepy as pp
import pygeon as pg

import sympy as sp

from math import ceil, floor, log10, exp

/opt/conda/lib/python3.11/site-packages/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


For a manual import:

import sys
sys.path.append('/path/to/repository')

In [3]:
from utilities.assembly_utilities import find_ordering, experimental_local_A, experimental_local_Mh, transoform_nodal_func_to_physical_element
from richards.model_params import Model_Data

### Parameters

In [4]:
# Set the maximum number of iterations of the non-linear solver
K = 500

# L-scheme parameter
L = 3.501e-2 #0.1

# Set the mesh refinment
N = 10

# Set the number of steps (excluding the initial condition)
num_steps = 27

quad_order = 3

# Simulation time length
T = num_steps/48

# Time switch conditions (for the boundary condition)
dt_D = 3/48

# Fluid density
rho = 1000

# Relative and absolute tolerances for the non-linear solver
abs_tol = 1e-10
rel_tol = 1e-5

# Domain tolerance
domain_tolerance = 1 / (10 * N)

# Output directory
output_directory = 'lagrange_hard_coupling'

In [5]:
def initial_h(x): return 1

In [6]:
# Van Genuchten model parameters ( relative permeability model )
model_data = Model_Data(theta_r=0.131, theta_s=0.396, alpha=0.423, n=2.06, K_s=4.96e-2, T=T, num_steps=num_steps)

In [7]:
# Time step
dt   = (T-0)/num_steps

### Mesh

In [8]:
bottom_grid =   pp.StructuredTriangleGrid([2*N, N], [2, 1])
bottom_grid.compute_geometry()

In [9]:
top_grid    = pp.StructuredTriangleGrid([2*N, 2*N], [2, 2])
top_grid.compute_geometry()

In [10]:
boundary_grid, boundary_face_map, boundary_node_map  = pp.partition.extract_subgrid(bottom_grid, 
                                                                                    bottom_grid.face_centers[1, :] == 1, 
                                                                                    faces=True)

### Discretization

In [11]:
key = "flow"

In [12]:
top_field = pg.Lagrange1(key)
top_data = {}

top_dof =  top_field.ndof( top_grid )

In [13]:
bottom_field = pg.Lagrange1(key)
bottom_data = {}

bottom_dof = bottom_field.ndof( bottom_grid )

In [14]:
boundary_field = pg.Lagrange1(key)
boundary_data = {}

boundary_dof = boundary_field.ndof( boundary_grid )

### Restrictor Matrices

In [15]:
bottom_delete_boundary_dof = np.zeros( shape=(bottom_dof - boundary_dof, bottom_dof) )
bottom_delete_boundary_dof[np.arange(bottom_dof - boundary_dof), bottom_grid.nodes[1, :] < 1] = 1
bottom_delete_boundary_dof = sps.csr_matrix(bottom_delete_boundary_dof)

In [16]:
bottom_isolate_boundary_dof = np.zeros(shape=(boundary_dof, bottom_dof))
bottom_isolate_boundary_dof[np.arange(boundary_dof), bottom_grid.nodes[1, :] == 1] = 1
bottom_isolate_boundary_dof = sps.csr_matrix(bottom_isolate_boundary_dof)

In [17]:
top_delete_boundary_dof = np.zeros( shape=(top_dof - boundary_dof, top_dof) )
top_delete_boundary_dof[np.arange(top_dof - boundary_dof), top_grid.nodes[1, :] > 0] = 1
top_delete_boundary_dof = sps.csr_matrix(top_delete_boundary_dof)

In [18]:
top_isolate_boundary_dof = np.zeros(shape=(boundary_dof, top_dof))
top_isolate_boundary_dof[np.arange(boundary_dof), top_grid.nodes[1, :] == 0] = 1
top_isolate_boundary_dof = sps.csr_matrix(top_isolate_boundary_dof)

### BCs

In [19]:
bot_dirichlet_flag  = np.array(np.logical_and( bottom_grid.nodes[0, :] == 2, bottom_grid.nodes[1, :] <= 1 ), dtype=bool)
bot_dirichlet_value = np.array(bot_dirichlet_flag, dtype=float)

In [20]:
top_dirichlet_flag  = np.array(np.logical_and( top_grid.nodes[1, :] == 2, top_grid.nodes[0, :] <= 1 ), dtype=bool)
top_dirichlet_value = lambda t: np.array(top_dirichlet_flag, dtype=float) * min( 6.2, 1 + 5.2 * t / dt_D )

In [21]:
dirichlet_flag  = np.hstack(( bot_dirichlet_flag, top_dirichlet_flag, np.zeros(shape=boundary_dof, dtype=bool)))
dirichlet_value = lambda t: np.hstack(( bot_dirichlet_value, top_dirichlet_value(t), np.zeros(shape=boundary_dof, dtype=bool)))

### Matrix Assembly

##### Projection matrices

In [22]:
top_proj    = top_field.eval_at_cell_centers( top_grid )
bottom_proj = bottom_field.eval_at_cell_centers( bottom_grid )

##### Stifness

In [23]:
def bottom_stifness_gen():
    size = np.power(bottom_grid.dim + 1, 2) * bottom_grid.num_cells
    rows_I = np.empty(size, dtype=int)
    cols_J = np.empty(size, dtype=int)
    data_IJ = np.empty(size)
    idx = 0

    _, _, _, _, _, node_coords = pp.map_geometry.map_grid(bottom_grid)

    # Allocate the data to store matrix entries, that's the most efficient
    # way to create a sparse matrix.

    cell_nodes = bottom_grid.cell_nodes()

    for c in np.arange(bottom_grid.num_cells):
        # For the current cell retrieve its nodes
        loc = slice(cell_nodes.indptr[c], cell_nodes.indptr[c + 1])

        nodes_loc = cell_nodes.indices[loc]
        coord_loc = node_coords[:, nodes_loc]

        # Compute the stiff-H1 local matrix

        A = experimental_local_A(coord_loc, lambda x,y: model_data.K_s * np.eye(2), quad_order)
        
        # Save values for stiff-H1 local matrix in the global structure
        cols = np.tile(nodes_loc, (nodes_loc.size, 1))

        loc_idx = slice(idx, idx + cols.size)
        rows_I[loc_idx] = cols.T.ravel()
        cols_J[loc_idx] = cols.ravel()
        data_IJ[loc_idx] = A.ravel()
        idx += cols.size

    # Construct the global matrices
    return sps.csc_matrix((data_IJ, (rows_I, cols_J)))

In [24]:
def top_stifness_gen(h_dofs):
    size = np.power(top_grid.dim + 1, 2) * top_grid.num_cells
    rows_I = np.empty(size, dtype=int)
    cols_J = np.empty(size, dtype=int)
    data_IJ = np.empty(size)
    idx = 0

    _, _, _, _, _, node_coords = pp.map_geometry.map_grid(top_grid)

    # Allocate the data to store matrix entries, that's the most efficient
    # way to create a sparse matrix.

    cell_nodes = top_grid.cell_nodes()

    for c in np.arange(top_grid.num_cells):
        # For the current cell retrieve its nodes
        loc = slice(cell_nodes.indptr[c], cell_nodes.indptr[c + 1])

        nodes_loc = cell_nodes.indices[loc]
        coord_loc = node_coords[:, nodes_loc]

        # Compute the stiff-H1 local matrix

        h_func = transoform_nodal_func_to_physical_element(h_dofs[nodes_loc], coord_loc)
        A = experimental_local_A(coord_loc, lambda x,y: model_data.hydraulic_conductivity_coefficient(h_func(x,y), 1+y) * np.eye(2), quad_order)
        
        # Save values for stiff-H1 local matrix in the global structure
        cols = np.tile(nodes_loc, (nodes_loc.size, 1))

        loc_idx = slice(idx, idx + cols.size)
        rows_I[loc_idx] = cols.T.ravel()
        cols_J[loc_idx] = cols.ravel()
        data_IJ[loc_idx] = A.ravel()
        idx += cols.size

    # Construct the global matrices
    return sps.csc_matrix((data_IJ, (rows_I, cols_J)))

##### Mass

In [25]:
def bottom_mass_gen():
    size = np.power(bottom_grid.dim + 1, 2) * bottom_grid.num_cells
    rows_I = np.empty(size, dtype=int)
    cols_J = np.empty(size, dtype=int)
    data_IJ = np.empty(size)
    idx = 0

    _, _, _, _, _, node_coords = pp.map_geometry.map_grid(bottom_grid)

    # Allocate the data to store matrix entries, that's the most efficient
    # way to create a sparse matrix.

    cell_nodes = bottom_grid.cell_nodes()

    for c in np.arange(bottom_grid.num_cells):
        # For the current cell retrieve its nodes
        loc = slice(cell_nodes.indptr[c], cell_nodes.indptr[c + 1])

        nodes_loc = cell_nodes.indices[loc]
        coord_loc = node_coords[:, nodes_loc]

        # Compute the stiff-H1 local matrix
        A = experimental_local_Mh(coord_loc, lambda x,y: 1, quad_order)

        # Save values for stiff-H1 local matrix in the global structure
        cols = np.tile(nodes_loc, (nodes_loc.size, 1))

        loc_idx = slice(idx, idx + cols.size)
        rows_I[loc_idx] = cols.T.ravel()
        cols_J[loc_idx] = cols.ravel()
        data_IJ[loc_idx] = A.ravel()
        idx += cols.size

    # Construct the global matrices
    return sps.csc_matrix((data_IJ, (rows_I, cols_J)))

In [26]:
def top_mass_gen():
    size = np.power(top_grid.dim + 1, 2) * top_grid.num_cells
    rows_I = np.empty(size, dtype=int)
    cols_J = np.empty(size, dtype=int)
    data_IJ = np.empty(size)
    idx = 0

    _, _, _, _, _, node_coords = pp.map_geometry.map_grid(top_grid)

    # Allocate the data to store matrix entries, that's the most efficient
    # way to create a sparse matrix.

    cell_nodes = top_grid.cell_nodes()

    for c in np.arange(top_grid.num_cells):
        # For the current cell retrieve its nodes
        loc = slice(cell_nodes.indptr[c], cell_nodes.indptr[c + 1])

        nodes_loc = cell_nodes.indices[loc]
        coord_loc = node_coords[:, nodes_loc]

        # Compute the stiff-H1 local matrix
        A = experimental_local_Mh(coord_loc, lambda x,y: 1, quad_order)

        # Save values for stiff-H1 local matrix in the global structure
        cols = np.tile(nodes_loc, (nodes_loc.size, 1))

        loc_idx = slice(idx, idx + cols.size)
        rows_I[loc_idx] = cols.T.ravel()
        cols_J[loc_idx] = cols.ravel()
        data_IJ[loc_idx] = A.ravel()
        idx += cols.size

    # Construct the global matrices
    return sps.csc_matrix((data_IJ, (rows_I, cols_J)))

### Solving stage

In [27]:
# assemble initial solution
initial_solution = np.zeros(top_dof + bottom_dof + boundary_dof)

In [28]:
bottom_mask = np.zeros_like(initial_solution, dtype=bool)
bottom_mask[ : (bottom_dof) ] = True

top_mask = np.zeros_like(initial_solution, dtype=bool)
top_mask[(bottom_dof) : (bottom_dof + top_dof)] = True

lambda_mask = np.zeros_like(bottom_mask, dtype=bool)
lambda_mask[(bottom_dof + top_dof):] = True

In [29]:
initial_solution[top_mask]    =    top_field.interpolate(   top_grid, lambda x: 1)
initial_solution[bottom_mask] = bottom_field.interpolate(bottom_grid, lambda x: 1)

In [30]:
top_saver    = pp.Exporter(top_grid,    "top_sol", folder_name=output_directory)
bottom_saver = pp.Exporter(bottom_grid, "bottom_sol", folder_name=output_directory)

def save_step(current_sol, step):
    ins = list()

    ins.append((top_grid, "cell_h", top_proj @ current_sol[top_mask]))
    ins.append((top_grid, "cell_p", top_proj @ (current_sol[top_mask] - 1 - top_grid.nodes[1, :])))
    top_saver.write_vtu(ins, time_step=step)

    ins = list()

    ins.append((bottom_grid, "cell_h", bottom_proj @ current_sol[bottom_mask]))
    ins.append((bottom_grid, "cell_p", bottom_proj @ (current_sol[bottom_mask] - bottom_grid.nodes[1, :])))
    bottom_saver.write_vtu(ins, time_step=step)

In [31]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [32]:
# solve the problem

sol = [initial_solution]

t = 0

save_step(sol[-1], 0)

In [33]:
csv_base = os.path.join(output_directory, 'csv')

if os.path.exists(csv_base):
    shutil.rmtree(csv_base)
    
os.mkdir(csv_base)

In [34]:
def bottom_normal_continuity_matrix():
    data = []
    cols = []
    rows = []
    _, _, _, _, _, node_coords = pp.map_geometry.map_grid(bottom_grid)

    # Allocate the data to store matrix entries, that's the most efficient
    # way to create a sparse matrix.

    cell_nodes = bottom_grid.cell_nodes()

    for c in np.arange(bottom_grid.num_cells):
        # For the current cell retrieve its nodes
        loc = slice(cell_nodes.indptr[c], cell_nodes.indptr[c + 1])

        nodes_loc = cell_nodes.indices[loc]
        coord_loc = node_coords[:, nodes_loc]

        if np.abs(np.max(coord_loc[1, :]) - 1) < 1e-7:
            
            ordering, m = find_ordering(coord_loc)

            if m < 0:
                eta_cell = np.max(np.where( boundary_grid.nodes[0, :] < bottom_grid.cell_centers[0, c] ))

                ordered_coords = coord_loc[:, ordering]

                x0 = ordered_coords[:, 0]
                x1 = ordered_coords[:, 1]
                x2 = ordered_coords[:, 2]
                
                q_funcs = [model_data.K_s * np.array([-1/(x2[0] - x0[0]), -1/(x1[1] - x0[1])]), 
                           model_data.K_s * np.array([                 0,  1/(x1[1] - x0[1])]), 
                           model_data.K_s * np.array([ 1/(x2[0] - x0[0]),                  0])]
            
                for i in range(3):
                    rows.append(eta_cell)
                    cols.append(nodes_loc[ ordering[i] ])
                    data.append(q_funcs[i].T @ np.array([0, 1]) * (x2[0] - x0[0]) / 2)

                    rows.append(eta_cell+1)
                    cols.append(nodes_loc[ ordering[i] ])
                    data.append(q_funcs[i].T @ np.array([0, 1]) * (x2[0] - x0[0]) / 2)

    
    return sps.coo_matrix((np.array(data), (np.array(rows), np.array(cols))), shape=(boundary_dof, bottom_grid.num_nodes))

In [35]:
def top_normal_continuity_matrix(h_dofs):
    data = []
    cols = []
    rows = []
    _, _, _, _, _, node_coords = pp.map_geometry.map_grid(top_grid)

    # Allocate the data to store matrix entries, that's the most efficient
    # way to create a sparse matrix.

    cell_nodes = top_grid.cell_nodes()

    for c in np.arange(top_grid.num_cells):
        # For the current cell retrieve its nodes
        loc = slice(cell_nodes.indptr[c], cell_nodes.indptr[c + 1])

        nodes_loc = cell_nodes.indices[loc]
        coord_loc = node_coords[:, nodes_loc]

        if np.abs(np.min(coord_loc[1, :]) - 0) < 1e-7:
            ordering, m = find_ordering(coord_loc)

            if m > 0:
                eta_cell = np.max(np.where( boundary_grid.nodes[0, :] < top_grid.cell_centers[0, c] ))

                ordered_coords = coord_loc[:, ordering]

                h_ord = h_dofs[nodes_loc][ordering]

                x0 = ordered_coords[:, 0]
                x1 = ordered_coords[:, 1]
                x2 = ordered_coords[:, 2]

                h_mean = (h_ord[0] + h_ord[2]) / 2

                q_funcs = [model_data.hydraulic_conductivity_coefficient(h_mean, 1) * np.array([-1/(x2[0] - x0[0]), -1/(x1[1] - x0[1])]), 
                           model_data.hydraulic_conductivity_coefficient(h_mean, 1) * np.array([                 0,  1/(x1[1] - x0[1])]), 
                           model_data.hydraulic_conductivity_coefficient(h_mean, 1) * np.array([ 1/(x2[0] - x0[0]),                  0])]
            
                for i in range(3):
                    rows.append(eta_cell)
                    cols.append(nodes_loc[ ordering[i] ])
                    data.append(q_funcs[i].T @ np.array([0, 1]) * (x2[0] - x0[0]) / 2)

                    rows.append(eta_cell+1)
                    cols.append(nodes_loc[ ordering[i] ])
                    data.append(q_funcs[i].T @ np.array([0, 1]) * (x2[0] - x0[0]) / 2)

    
    return sps.coo_matrix((np.array(data), (np.array(rows), np.array(cols))), shape=(boundary_dof, top_grid.num_nodes))

In [36]:
top_mass = top_mass_gen()
bottom_mass_gen = bottom_mass_gen()

In [37]:
bottom_stifness = bottom_stifness_gen()

In [38]:
bottom_vel = bottom_normal_continuity_matrix()

In [39]:
# Time Loop
for step in range(1, ceil(T/dt) + 1):
    current_time = step * dt
    print('Time ' + str(round(current_time, 5)))
    
    time_rhs = np.zeros_like(sol[-1])

    time_rhs[top_mask] += top_mass @ model_data.theta(sol[-1][top_mask], 1 + top_grid.nodes[1,:]) / dt

    prev = sol[-1]

    for k in range(K):
        rhs = time_rhs.copy()

        rhs[top_mask] += L * top_mass @ prev[top_mask] / dt
        rhs[top_mask] -= top_mass @ model_data.theta(prev[top_mask], 1 + top_grid.nodes[1,:]) / dt

        bottom_mat = bottom_stifness
        top_stifness = top_stifness_gen(prev[top_mask])

        top_mat = top_stifness + L / dt * top_mass
        top_vel = top_normal_continuity_matrix(prev[top_mask])

        spp = sps.bmat([[                 bottom_mat,                      None, bottom_isolate_boundary_dof.T],
                        [                       None,                   top_mat,   -top_isolate_boundary_dof.T],
                        [bottom_isolate_boundary_dof, -top_isolate_boundary_dof,                          None]], format='csc')
        
        ls = pg.LinearSystem(spp, rhs)

        #print(spp, rhs)

        ls.flag_ess_bc(dirichlet_flag, dirichlet_value(current_time))

        current = ls.solve()

        # Check if we have reached convergence
        rel_err_psi  = np.sqrt( (current - prev).T @ (current - prev) )
        abs_err_prev = np.sqrt( prev.T @ prev )

        # Log message with error and current iteration
        print('Iteration #' + format(k+1, '0' + str(ceil(log10(K)) + 1) + 'd') + ', error L2 relative psi: ' 
              + format(rel_err_psi, str(5 + ceil(log10(1 / abs_tol)) + 4) + '.' + str(ceil(log10(1 / abs_tol)) + 4) + 'f') )
        
        if rel_err_psi > abs_tol + rel_tol * abs_err_prev:
            prev = current.copy()
        else:
            break

    sol.append( current )

    save_step(sol[-1], step)
    print('')    

Time 0.02083
Iteration #0001, error L2 relative psi:    5.81082990976450


<lambdifygenerated-1>:2: RuntimeWarning: invalid value encountered in power
  return select([less(h, z),True], [0.265*(0.1699265174169*(-h + z)**2.06 + 1)**(-0.514563106796116) + 0.131,0.396], default=nan)
<lambdifygenerated-2>:2: RuntimeWarning: invalid value encountered in scalar power
  return select([less(h, z),True], [0.0496*(1 - (1 - 1.0*(0.1699265174169*(-h + z)**2.06 + 1)**(-1.0))**0.514563106796116)**2*(0.1699265174169*(-h + z)**2.06 + 1)**(-0.257281553398058),0.0496], default=nan)


Iteration #0002, error L2 relative psi:    1.85220640097616
Iteration #0003, error L2 relative psi:    0.27731022909162
Iteration #0004, error L2 relative psi:    0.01844888899250
Iteration #0005, error L2 relative psi:    0.00339835609219
Iteration #0006, error L2 relative psi:    0.00049269964707
Iteration #0007, error L2 relative psi:    0.00004047629196

Time 0.04167
Iteration #0001, error L2 relative psi:    7.50744633618925
Iteration #0002, error L2 relative psi:    2.75617895132764
Iteration #0003, error L2 relative psi:    0.94191865636499
Iteration #0004, error L2 relative psi:    0.21402123241571
Iteration #0005, error L2 relative psi:    0.07967841800755
Iteration #0006, error L2 relative psi:    0.02452486856682
Iteration #0007, error L2 relative psi:    0.00558836153812
Iteration #0008, error L2 relative psi:    0.00198684563946
Iteration #0009, error L2 relative psi:    0.00051285066009
Iteration #0010, error L2 relative psi:    0.00014826896506

Time 0.0625
Iteration #00

In [40]:
prev

array([ 2.80801741e+00,  2.80357888e+00,  2.79025095e+00,  2.76799685e+00,
        2.73675607e+00,  2.69644575e+00,  2.64696313e+00,  2.58818958e+00,
        2.51999700e+00,  2.44225753e+00,  2.35485799e+00,  2.25772054e+00,
        2.15083118e+00,  2.03427754e+00,  1.90829659e+00,  1.77333056e+00,
        1.63008662e+00,  1.47959031e+00,  1.32321773e+00,  1.16268795e+00,
        1.00000000e+00,  2.81245594e+00,  2.80802357e+00,  2.79471403e+00,
        2.77249020e+00,  2.74129084e+00,  2.70103190e+00,  2.65160859e+00,
        2.59289910e+00,  2.52477044e+00,  2.44708756e+00,  2.35972695e+00,
        2.26259650e+00,  2.15566331e+00,  2.03899121e+00,  1.91278912e+00,
        1.77746951e+00,  1.63371281e+00,  1.48252845e+00,  1.32529633e+00,
        1.16376704e+00,  1.00000000e+00,  2.82575921e+00,  2.82134544e+00,
        2.80809142e+00,  2.78595905e+00,  2.75488521e+00,  2.71478241e+00,
        2.66554023e+00,  2.60702778e+00,  2.53909812e+00,  2.46159531e+00,
        2.37436574e+00,  